### Sklearn Pipeline
Doing preprocessing the data manually for test cases is both arduos and time-consuming. In fact, it will not give you proper result if something goes wrong. As a result, an effective solution is sklearn's pipeline.

<b style = 'color:red'>We will also see, how to use feature selection during pipeline.</b>

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
328,329,1,3,"Goldsmith, Mrs. Frank John (Emily Alice Brown)",female,31.0,1,1,363291,20.5250,NaN,S
786,787,1,3,"Sjoblom, Miss. Anna Sofia",female,18.0,0,0,3101265,7.4958,NaN,S


In [4]:
"""
Dropping those columns which are not neccessary for our analysis.
"""
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [5]:
df.sample(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
520,1,1,female,30.0,0,0,93.50,S
330,1,3,female,NaN,2,0,23.25,Q
582,0,2,male,54.0,0,0,26.00,S


In [6]:
"""
Step 1:
    Train_Test_Split()
"""
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,
                                                 random_state=42)

In [7]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [8]:
y_train.head()

331    0
733    0
382    0
704    0
813    0
Name: Survived, dtype: int64

In [9]:
"""
Step 2:
    Imputation Transformer.
    
    We use index value instead of column name because the output is given in a numpy array rather than a dataframe. Hence, 
    we use index value and not column names so that in pipeline everything runs smoothly because numpy array doesn't have 
    column names.
    
"""

trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),    
    ('impute_embarked',SimpleImputer(strategy = 'most_frequent'),[6])
],remainder='passthrough')

In [10]:
"""
OneHot Encoding in ColumnTransformer.
"""

trf2 = ColumnTransformer(transformers=[
    ('ohe_sex_embarked',OneHotEncoder(sparse = False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [11]:
"""
Scaling
"""

trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [12]:
"""
Feature Selection.
"""

trf4 = SelectKBest(score_func=chi2,k=8)

In [13]:
"""
train the model.
"""

trf5 = DecisionTreeClassifier()

### Creating a Pipeline Object

In [14]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

### Pipeline vs make_pipeline
<b style = 'color:orange'>Pipeline</b> requires you to give names of the steps but in <b style = 'color:orange'>make_pipeline()</b> you don't need to do this. Same goes for <b style = 'color:green'>ColumnTransformer vs make_column_transformer</b>. Though make_pipeline has easier syntax, we should use Pipeline class because it gives us more <b style = 'color:red'>readability.</b> Easy for <b style = 'color:red'>debugging.</b>


In [121]:
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

### Exploring the pipeline

In [15]:
"""
To display pipeline, you need to write this code.
"""

from sklearn import set_config
set_config(display='diagram')

### Visualizing the pipeline

In [17]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002425C4ECE50>)),
                ('trf5', DecisionTreeClassifier())])

In [19]:
"""
Use the attribute named_steps or steps to inspect estimators within the pipeline. 
"""
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x000002425C4ECE50>),
 'trf5': DecisionTreeClassifier()}

In [20]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

### Backtracking pipeline when necessary.

Good for <b style = 'color:orange'>Debugging.</b> You can go to any transformer and check for parameters.

In [27]:
"""
Steps to find the mean of first simpleImputer
    pipe.named_steps['trf1']
    pipe.names_steps['trf1'].transformers_
    pipe.names_steps['trf1'].transformers_[0]
    pipe.names_steps['trf1'].transformers_[0][1]
    pipe.names_steps['trf1'].transformers_[0][1].statistics_
"""
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [33]:
"""
Find out which one of the embarked class is most frequent?
"""
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

#### Predict

In [37]:
y_pred = pipe.predict(X_test)

In [35]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

### Cross Validation using Pipeline

In [38]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv = 5,scoring='accuracy').mean()

0.6391214419383433

### GridSearch using Pipeline

In [45]:
params = {
    'trf5__max_depth' : [1,2,3,4,5,None]
}

In [46]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000002425C4ECE50>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [47]:
grid.best_score_

0.6391214419383433

In [48]:
grid.best_params_

{'trf5__max_depth': 2}

### Exporting the Pipeline

What if you need to use this pipeline in production.

In [50]:
import pickle
pickle.dump(pipe,open('model/pipe.pkl','wb'))